In [101]:
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib
import matplotlib.pyplot as plt
import random


Load in the Data

In [102]:
games_df = pd.read_csv("C:/Users/jesse/Desktop/CSE141L/COGS118BProject/Data/nfl-big-data-bowl-2024/games.csv")
games_df

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore
0,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31
1,2022091100,2022,1,09/11/2022,13:00:00,ATL,NO,26,27
2,2022091101,2022,1,09/11/2022,13:00:00,CAR,CLE,24,26
3,2022091102,2022,1,09/11/2022,13:00:00,CHI,SF,19,10
4,2022091103,2022,1,09/11/2022,13:00:00,CIN,PIT,20,23
...,...,...,...,...,...,...,...,...,...
131,2022110607,2022,9,11/06/2022,13:00:00,WAS,MIN,17,20
132,2022110608,2022,9,11/06/2022,16:05:00,ARI,SEA,21,31
133,2022110609,2022,9,11/06/2022,16:25:00,TB,LA,16,13
134,2022110610,2022,9,11/06/2022,20:20:00,KC,TEN,20,17


In [115]:
plays_df = pd.read_csv("C:/Users/jesse/Desktop/CSE141L/COGS118BProject/Data/nfl-big-data-bowl-2024/plays.csv")
plays_df = plays_df[["playDescription" , "quarter", "down", "yardsToGo", "possessionTeam","defensiveTeam","absoluteYardlineNumber", "gameClock", "preSnapHomeScore","preSnapVisitorScore","passResult", "offenseFormation", "defendersInTheBox"]]
plays_df.shape

(12486, 13)

Seperating plays by team and creating a test and train dataset for each team(later we can do analysis on which teams we can predict the best)

In [116]:
teams = plays_df.possessionTeam.unique()
#empty dictionary of team names
plays_df_by_team_test = {x : pd.DataFrame() for x in teams}
plays_df_by_team_train = {x : pd.DataFrame() for x in teams}
#fills dictionary with all plays according to possesion team
for team in plays_df_by_team_test.keys():
    plays_df_by_team_test[team] = plays_df[:][plays_df.possessionTeam == team]
#seperate into train and test sets seperated by team
for team in plays_df_by_team_test.keys():  
    rows = len(plays_df_by_team_test[team])
    rand_idx = np.random.randint(0, rows,size = int(rows/10))
    plays_df_by_team_train[team] = plays_df_by_team_test[team].drop(plays_df_by_team_test[team].index[rand_idx])
    plays_df_by_team_test[team] = plays_df_by_team_test[team].iloc[rand_idx]

plays_df_by_team_test['ATL']

,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,absoluteYardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,offenseFormation,defendersInTheBox
1064,"(3:12) (No Huddle, Shotgun) M.Mariota scramble...",3,2,8,ATL,CAR,74,3:12,14,13,R,SHOTGUN,7.0
4041,(2:26) (Shotgun) M.Mariota pass deep right to ...,4,1,15,ATL,CAR,57,2:26,24,28,C,SHOTGUN,6.0
5319,(5:27) (Shotgun) T.Allgeier left end pushed ob...,3,1,10,ATL,TB,53,5:27,21,0,NaN,PISTOL,7.0
10049,(12:17) (Shotgun) M.Mariota pass short left to...,1,2,7,ATL,SF,36,12:17,0,0,C,PISTOL,6.0
1506,(1:41) C.Patterson right end to NO 43 for 7 ya...,3,1,10,ATL,NO,60,1:41,23,10,NaN,SINGLEBACK,6.0
3025,(2:39) T.Allgeier left end to CLV 27 for no ga...,4,3,3,ATL,CLE,83,2:39,20,20,NaN,SINGLEBACK,8.0
3713,(1:54) (Shotgun) T.Allgeier right guard to SF ...,2,1,10,ATL,SF,92,1:54,14,14,NaN,PISTOL,6.0
10300,(8:34) (Shotgun) M.Mariota pass short left to ...,1,3,10,ATL,NO,31,8:34,3,0,C,EMPTY,5.0
10475,"(8:18) C.Patterson right tackle for 17 yards, ...",2,1,10,ATL,SEA,93,8:18,10,10,NaN,I_FORM,7.0
6534,"(11:01) (No Huddle, Shotgun) C.Huntley left ta...",4,1,10,ATL,CAR,60,11:01,21,21,NaN,PISTOL,6.0


In [ ]:
tackles_df = pd.read_csv("C:/Users/jesse/Desktop/CSE141L/COGS118BProject/Data/nfl-big-data-bowl-2024/tackles.csv")
tackles_df

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle
0,2022090800,101,42816,1,0,0,0
1,2022090800,393,46232,1,0,0,0
2,2022090800,486,40166,1,0,0,0
3,2022090800,646,47939,1,0,0,0
4,2022090800,818,40107,1,0,0,0
...,...,...,...,...,...,...,...
17421,2022091113,2494,43533,0,0,0,1
17422,2022092502,3510,42406,0,0,0,1
17423,2022091113,3642,43478,0,0,0,1
17424,2022091901,3578,42431,0,0,0,1
